In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
#!ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432

In [2]:
conn = psycopg2.connect(database = "datamyne20170707", user = "ram22")
cur = conn.cursor()

In [3]:
cur.execute("""
drop table if exists puma;
create table puma as
select shipper_unified, shipper_declared, consignee_unified, consignee_declared, shipper_d_b_business_name, consignee_d_b_business_name
from billoflading 
where shipper_d_b_business_name='Puma North America, Inc.' or consignee_d_b_business_name='Puma North America, Inc.' 
or shipper_declared ~* '.*puma.*' or consignee_declared ~* '.*puma.*'
or shipper_unified ~* '.*puma.*' or consignee_unified ~* '.*puma.*';

COPY puma to '/home/ram22/dataDrive/dataProjects/postgresql/tempOutput/puma.csv' WITH CSV HEADER;
""")    

In [4]:
cur.execute("""
drop table if exists pumaDBonly;
create table pumaDBonly as
select *
from billoflading 
where consignee_d_b_business_name='Puma North America, Inc.';

COPY pumaDBonly to '/home/ram22/dataDrive/dataProjects/postgresql/tempOutput/pumaDBonly.csv' WITH CSV HEADER;
""")

In [5]:
conn.commit()
conn.close()

In [3]:
puma = pd.read_csv('puma.csv')

In [7]:
puma.consignee_unified.value_counts()

PUMA NORTH AMERICA INC, MA                 1983
PUMA NORTH AMERICA INC, WO                  970
COBRA GOLF INC DBA COBRA PUMA GOLF, WO      126
PUMA KIDS APPAREL NA LLC, NY                 92
LAZYONEINCRICHMOND APPAREL GROUP, UT          7
COBRA GOLF INC DBA COBRA PUMA GOLFU, CA       6
PUMA NORTH AMERICA C O NYK, CA                5
PUMA INTERNATIONAL TRADING GMBH, MA           4
CRICKET MERCHANT LLC, IL                      2
M S LAZYONE INC, UT                           1
COBRA PUMA GOLF, CA                           1
Name: consignee_unified, dtype: int64

In [8]:
1983+970+92+5+4

3054

In [9]:
puma.consignee_declared.value_counts()

PUMA NORTH AMERICA INC                 2394
PUMA NORTH AMERICA, INC.                209
PUMA NORTH AMERICA                      203
COBRA GOLF INC DBA COBRA PUMA GOLF      124
PUMA NORTH AMERICA INC.                  77
PUMA KIDS APPAREL LLC                    67
PUMA NORTH AMERICA, INC                  23
PUMA  NORTH AMERICA                      18
PUMA NORTH AMERICA.                      17
PUMA NORTH AMERICA,                      15
PUMA KIDS APPAREL                        14
PUMA KIDS APPAREL N A LLC                 8
COBRA GOLF INC DBA COBRA PUMA GOLFU       6
PUMA INTERNATIONAL TRADING GMBH           4
LAZYONE, INC,                             3
PUMA KIDS APPAREL NA LLC                  3
LAZYONE INC                               3
CRICKET MERCHANT LLC                      2
COBRA GOLF INC DBA COBRA PUMA             2
PUMA  NORTH  AMERICA                      1
COBRA PUMA GOLF                           1
PUMA NORTH AMERACA                        1
LAZY ONE INC                    

In [10]:
2394+209+203+77+67+23+18+17+15+14+8+4+3+1+1

3054

In [12]:
puma.consignee_d_b_business_name.value_counts(dropna=False)

Puma North America, Inc.        1869
NaN                             1318
Richmond Apparel Group, Inc.       7
Cricket Merchant                   2
Cobra Golf Incorporated            1
Name: consignee_d_b_business_name, dtype: int64

In [13]:
1869+1318

3187

In [14]:
puma.shipper_declared[puma.consignee_d_b_business_name=='Richmond Apparel Group, Inc.'].value_counts(dropna=False)

PUMA HOSIERY MILL        6
M S PUMA HOSIERY MILL    1
Name: shipper_declared, dtype: int64

In [15]:
#PUMA HOSIERY MILL IS PUMA!
puma.shipper_unified[puma.consignee_d_b_business_name=='Richmond Apparel Group, Inc.'].value_counts(dropna=False)

PUMA HOSIERY MILL (IN)    7
Name: shipper_unified, dtype: int64

In [16]:
puma.shipper_declared[puma.consignee_d_b_business_name=='Cricket Merchant'].value_counts(dropna=False)

PUMA SPORTS INDIA PVT LTD    2
Name: shipper_declared, dtype: int64

In [34]:
pumaDBonly = pd.read_csv('pumaDBonly.csv')

In [35]:
#name
pumaDBonly.consignee_d_b_business_name.value_counts(dropna=False)

Puma North America, Inc.    1869
Name: consignee_d_b_business_name, dtype: int64

In [31]:
#cityName

# declared zip code is either read incorrectly or useless
print(pumaDBonly.consignee_d_b_city_name.value_counts(dropna=False))
print(pumaDBonly.consignee_declared_city.value_counts(dropna=False))
print(pumaDBonly.consignee_declared_zip_code_city.value_counts(dropna=False))
print(pumaDBonly.consignee_d_b_db_city_code.value_counts(dropna=False))

print(pumaDBonly.consignee_d_b_postal_code.value_counts(dropna=False)) # 8 digits?

Westford    1869
Name: consignee_d_b_city_name, dtype: int64
WESTFORD, MA    1869
Name: consignee_declared_city, dtype: int64
WESTFORD, MA    1330
WESTFORD         539
Name: consignee_declared_zip_code_city, dtype: int64
8785    1869
Name: consignee_d_b_db_city_code, dtype: int64
18863616    1869
Name: consignee_d_b_postal_code, dtype: int64


In [32]:
#dateEstablished
print(pumaDBonly.consignee_d_b_year_started.value_counts(dropna=False))

1995    1869
Name: consignee_d_b_year_started, dtype: int64


In [33]:
#latitude, longitude
print(pumaDBonly.consignee_declared_zip_code_latitude.value_counts(dropna=False))
print(pumaDBonly.consignee_d_b_latitude.value_counts(dropna=False))
print(pumaDBonly.consignee_declared_zip_code_longitude.value_counts(dropna=False))
print(pumaDBonly.consignee_d_b_longitude.value_counts(dropna=False))

NaN           1330
 42.446396     539
Name: consignee_declared_zip_code_latitude, dtype: int64
42.561173    1869
Name: consignee_d_b_latitude, dtype: int64
NaN           1330
-71.459405     539
Name: consignee_declared_zip_code_longitude, dtype: int64
-71.424057    1869
Name: consignee_d_b_longitude, dtype: int64


In [37]:
#ownership
pumaDBonly.consignee_d_b_legal_status_name.value_counts(dropna=False)

CORPORATION    1869
Name: consignee_d_b_legal_status_name, dtype: int64

In [38]:
#primaryNAICS, primaryNAICSDescription
print(pumaDBonly.consignee_d_b_primary_naics_1_1_code.value_counts())
print(pumaDBonly.consignee_d_b_primary_naics_1_1_description.value_counts())

424320    1869
Name: consignee_d_b_primary_naics_1_1_code, dtype: int64
Men's and Boys' Clothing and Furnishings Merchant Wholesalers    1869
Name: consignee_d_b_primary_naics_1_1_description, dtype: int64


In [42]:
#salesVolume
pumaDBonly.consignee_d_b_sales_volume_us_dollars.value_counts()

481249487.0    1869
Name: consignee_d_b_sales_volume_us_dollars, dtype: int64

In [39]:
#sizeOfOrganization
pumaDBonly.consignee_d_b_employees_total.value_counts()

507    1869
Name: consignee_d_b_employees_total, dtype: int64

In [43]:
#stateProvinceName
print(pumaDBonly.consignee_declared_state.value_counts())
print(pumaDBonly.consignee_declared_state_abb.value_counts())
print(pumaDBonly.consignee_declared_zip_code_state.value_counts())
print(pumaDBonly.consignee_d_b_state_province_name.value_counts())
print(pumaDBonly.consignee_d_b_db_state_code.value_counts())

MASSACHUSETTS, MA    1869
Name: consignee_declared_state, dtype: int64
MA    1869
Name: consignee_declared_state_abb, dtype: int64
MA    539
Name: consignee_declared_zip_code_state, dtype: int64
Massachusetts    1869
Name: consignee_d_b_state_province_name, dtype: int64
41    1869
Name: consignee_d_b_db_state_code, dtype: int64


In [44]:
#streetAddress
print(pumaDBonly.consignee_d_b_street_address.value_counts())
print(pumaDBonly.consignee_d_b_street_address_2.value_counts())

10 Lyberty Way    1869
Name: consignee_d_b_street_address, dtype: int64
Series([], Name: consignee_d_b_street_address_2, dtype: int64)


In [47]:
#totalContainers
pumaDBonly.container_quantity = pd.to_numeric(pumaDBonly.container_quantity)
pumaDBonly.container_quantity.sum()

1486.1199999999978

In [48]:
#totalTEUS
pumaDBonly.calc_teus = pd.to_numeric(pumaDBonly.calc_teus)
pumaDBonly.calc_teus.sum()

2820.5000000000005

In [49]:
#inputVolume
#totalEstimatedTradeValue is the same as we have no exports
#
pumaDBonly.calc_value_usd = pd.to_numeric(pumaDBonly.calc_value_usd)
pumaDBonly.calc_value_usd.sum()

168613644.61999926

In [50]:
#topCountriesTradedWith: only imports
pumaDBonly.country_of_origin.value_counts()

INDONESIA      689
CHINA          372
CAMBODIA       324
VIETNAM        155
BANGLADESH      77
EL SALVADOR     58
MALAYSIA        48
PHILIPPINES     42
PAKISTAN        36
TAIWAN          22
MEXICO          19
INDIA            7
HONDURAS         4
MACAU            4
JORDAN           3
TURKEY           3
SINGAPORE        3
HONG KONG        2
SLOVAKIA         1
Name: country_of_origin, dtype: int64

In [53]:
#topSuppliers: only imports
print(pumaDBonly.shipper_declared.value_counts(dropna=False))
print(pumaDBonly.shipper_d_b_business_name.value_counts(dropna=False))

PT POU CHEN INDONESIA                  372
PT HORN MING INDONESIA                 233
DAMCO CAMBODIA LTD                     122
MAERSK VIETNAM LTD ON BEHALF OF         92
DAMCO (CAMBODIA) LTD ON BEHALF  OF      69
BUDDIES MACAO COMMERCIAL OFFSHORE       51
DAMCO (CAMBODIA) LTD ON BEHALF  O       51
HEADWAY CAPS INTERNATIONAL CO INC       44
BEAUTIFUL SPRING FOOTWEAR CO.,LTD       43
PT .BUILDYET INDONESIA                  35
TSG INTERNATIONAL LIMITED               34
KEENWAY INDUSTRIES SDN BHD              33
DIAMOND GROUP INTERNATIONAL LTD         32
KARNAPHULI SHOES INDS. LTD.             27
HUAXUN LOGISTICS SHENZHEN COMPANY L     27
BUDDIES (MACAO COMMERCIAL OFFSHORE)     26
CONFECCIONES DEL VALLE SA DE            24
DAMCO INTL BV C/O MAERSK VIETNAM LT     22
PUTIAN BAIHO FOOTWEAR COMPANY LIMIT     21
REGINA MIRACLE INTERNATIONAL (GROUP     19
BIHQ PTE LTD                            18
ALI TRADING CO. (PVT.) LTD.,            18
DA SHENG (BVI) INTL HOLDING LIMITED     17
GOODDALE LT